In [1]:
richtung_WS = 1 #Richtung des Wärmestroms, 0: Aufwärts, 1:Horizontal, 2:Abwärts
r = 4 # 精度小数点后几位
homogen = True # homogene Bauteile, True or False

list_d = [0.015,0.365,0.02,0.14,0.01] # unit [m]
list_Lambda = [1,0.81,1,0.035,1] # unit [W/mK]

In [2]:
if richtung_WS == 0:
    R_si = 0.1
    R_se = 0.04
elif richtung_WS == 1:
    R_si = 0.13
    R_se = 0.04
else:
    R_si = 0.17
    R_se = 0.04

In [3]:
# functions
## calculate Wärmewiderstand R with Dicke d and Lambda.
def dL2R(d,Lambda):
    R = d / Lambda
    R = round(R,r)
    return R

## calculate U value for homogene Bauteile
def homogene_bauteile_U(list_R):
    R_T = R_si + R_se + sum(list_R)
    U = 1 / R_T
    return U


In [4]:
# calculate R value
list_R = []

if len(list_d) != len(list_Lambda):
    print('ERROR: d and Lambda not match')
else:
    for i in range (0,len(list_d)):
        list_R.append(dL2R(list_d[i],list_Lambda[i]))

print(list_R)
print(R_si,R_se)

[0.015, 0.4506, 0.02, 4.0, 0.01]
0.13 0.04


In [5]:
homogene_bauteile_U(list_R)

0.21433470507544583